# Querying DataFrame and Handling Missing Values

**Learning Objectives**

* Ways to Handling Missing Data

* query dataframe using boolean masking



In this lecture we're going to talk about querying DataFrames. The first step in the process is to understand Boolean masking. Boolean masking is the heart of fast and efficient querying in numpy and pandas, and it's analogous to bit masking used in other areas of computational science. By the end of this lecture you'll understand how Boolean masking works, and how to apply this to a DataFrame to get out data you're interested in.

A Boolean mask is an array which can be of one dimension like a series, or two dimensions like a data frame, where each of the values in the array are either true or false. This array is essentially overlaid on top of the data structure that we're querying. And any cell aligned with the true value will be admitted into our final result, and any cell aligned with a false value will not.

In [1]:
# Let's start with an example and import our graduate admission dataset. First we'll bring in pandas
import pandas as pd
# Then we'll load in our CSV file
df = pd.read_csv('class_grades.csv', index_col=0)
# And let's look at the first few rows
df.head()


,Assignment,Tutorial,Midterm,TakeHome,Final
Prefix,,,,,
5,57.14,34.09,64.38,51.48,52.50
8,95.05,105.49,67.50,99.07,68.33
8,83.70,83.17,NaN,63.15,48.89
7,NaN,NaN,49.38,105.93,80.56
8,91.32,93.64,95.00,107.41,73.89


In [2]:
df.columns

Index(['Assignment', 'Tutorial   ', 'Midterm', 'TakeHome', 'Final     '], dtype='object')

 If we look at the output closely, we can see that there is actually a space right after "Tutorial" and a space right after "Final"

In [3]:
# And we'll clean up a couple of poorly named columns
df.columns = [x.lower().strip() for x in df.columns]
# And we'll take a look at the results
df.head()

,assignment,tutorial,midterm,takehome,final
Prefix,,,,,
5,57.14,34.09,64.38,51.48,52.50
8,95.05,105.49,67.50,99.07,68.33
8,83.70,83.17,NaN,63.15,48.89
7,NaN,NaN,49.38,105.93,80.56
8,91.32,93.64,95.00,107.41,73.89


In [4]:
df.columns

Index(['assignment', 'tutorial', 'midterm', 'takehome', 'final'], dtype='object')

# Data Manipulation

* Fixing the Prefix Column

In [5]:
# Generate a sequence of numbers from 1 to the number of rows in the DataFrame
import numpy as np
prefix_values = np.arange(1, len(df) + 1)
#change the name of the index column in your DataFrame:
df = df.rename_axis("Old_Prefix")


In [6]:
df

,assignment,tutorial,midterm,takehome,final
Old_Prefix,,,,,
5,57.14,34.09,64.38,51.48,52.50
8,95.05,105.49,67.50,99.07,68.33
8,83.70,83.17,NaN,63.15,48.89
7,NaN,NaN,49.38,105.93,80.56
8,91.32,93.64,95.00,107.41,73.89
...,...,...,...,...,...
8,NaN,103.71,45.00,93.52,61.94
7,NaN,80.54,41.25,93.70,39.72
8,89.94,102.77,87.50,90.74,87.78


In [7]:
df.shape

(99, 5)

In [8]:
# Assign the sequence to the 'Prefix' column
df['Serial_Numb'] = prefix_values

In [9]:
# Reset the index
df = df.reset_index()

# Drop the old 'Prefix' column (now a regular column)
df = df.drop(columns=['Old_Prefix'])  # Assuming the original 'Prefix' column is named 'Prefix' after resetting the index

# Rename the newly generated column from 'level_0' to 'Prefix'
#df = df.rename(columns={'level_0': 'Prefix'})
df = df.set_index("Serial_Numb")

In [10]:
df

,assignment,tutorial,midterm,takehome,final
Serial_Numb,,,,,
1,57.14,34.09,64.38,51.48,52.50
2,95.05,105.49,67.50,99.07,68.33
3,83.70,83.17,NaN,63.15,48.89
4,NaN,NaN,49.38,105.93,80.56
5,91.32,93.64,95.00,107.41,73.89
...,...,...,...,...,...
95,NaN,103.71,45.00,93.52,61.94
96,NaN,80.54,41.25,93.70,39.72
97,89.94,102.77,87.50,90.74,87.78


* Handling Missing values - check for missing values 

3 options:
- Delete
- Data Imputation - domain knowledge
- Provide a default value

In [11]:
mask=df.isnull()
#reates a Boolean mask indicating the location of null values,
#it doesn't directly provide a count of those nulls.
mask

,assignment,tutorial,midterm,takehome,final
Serial_Numb,,,,,
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,True,False,False
4,True,True,False,False,False
5,False,False,False,False,False
...,...,...,...,...,...
95,True,False,False,False,False
96,True,False,False,False,False
97,False,False,False,False,False


In [12]:
mask.head(15)

,assignment,tutorial,midterm,takehome,final
Serial_Numb,,,,,
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,True,False,False
4,True,True,False,False,False
5,False,False,False,False,False
6,False,False,False,False,False
7,False,False,False,False,False
8,False,False,False,True,False
9,False,False,False,False,False


In [13]:
mask.sum() #Applying .sum() to the Boolean mask (mask) calculates the sum of True values along each column. Since True is treated as 1 and False as 0, it essentially counts nulls per column.
#The result is a Series containing the null counts for each column.

assignment    7
tutorial      3
midterm       3
takehome      5
final         3
dtype: int64

In [14]:
total_nulls = mask.sum().sum()
#Applying .sum() again to the Series obtained in the previous step
#adds up all the column-wise null counts,
#giving you the total number of null values in the entire DataFrame.
#total_nulls = df.isnull().sum().sum()

In [15]:
total_nulls

np.int64(21)

In [16]:
# This can be useful for processing rows based on certain columns of data. Another useful operation is to be
# able to drop all of those rows which have any missing data, which can be done with the dropna() function.
df.dropna().head(15)

,assignment,tutorial,midterm,takehome,final
Serial_Numb,,,,,
1,57.14,34.09,64.38,51.48,52.50
2,95.05,105.49,67.50,99.07,68.33
5,91.32,93.64,95.00,107.41,73.89
6,95.00,92.58,93.12,97.78,68.06
7,95.05,102.99,56.25,99.07,50.00
9,84.26,93.10,47.50,18.52,50.83
10,90.10,97.55,51.25,88.89,63.61
11,80.44,90.20,75.00,91.48,39.72
13,97.16,103.71,72.50,93.52,63.33


In [17]:
# Note how the rows 4 and 12 (and several others) are now gone. One of the handy functions that Pandas has for
# working with missing values is the filling function, fillna(). This function takes a number or parameters.
# You could pass in a single value which is called a scalar value to change all of the missing data to one
# value. This isn't really applicable in this case, but it's a pretty common use case.

# So, if we wanted to fill all missing values with 0, we would use fillna
#df.fillna(0, inplace=True)
# The inplace attribute causes pandas to fill the values in line and does not return a copy of
# the dataframe, but instead modifies the dataframe that you have
# Usage Example:


import pandas as pd
import numpy as np

# Example DataFrame
data = {'A': [1, np.nan, 3],
        'B': [np.nan, 5, 6],
        'C': [7, 8, np.nan]}

example_df = pd.DataFrame(data)
print(example_df)

# Replacing NaN values with 199
example_df.fillna(199, inplace=True)

print(example_df)


     A    B    C
0  1.0  NaN  7.0
1  NaN  5.0  8.0
2  3.0  6.0  NaN
       A      B      C
0    1.0  199.0    7.0
1  199.0    5.0    8.0
2    3.0    6.0  199.0


* Data Manipulation

In [18]:
# Boolean masks are created by applying operators directly to the pandas Series or DataFrame objects.
# For instance, in our graduate admission dataset, we might be interested in seeing only those students
# that have a final grade higher than 70
# To build a Boolean mask for this query, we want to project the chance of admit column using the
# indexing operator and apply the greater than operator with a comparison value of 70. This is
# essentially broadcasting a comparison operator, greater than, with the results being returned as
# a Boolean Series. The resultant Series is indexed where the value of each cell is either True or False
# depending on whether a student has a chance of admit higher than 70
nPassed_Exam=df['final'] > 70.0
nPassed_Exam

Serial_Numb
1     False
2     False
3     False
4      True
5      True
      ...  
95    False
96    False
97     True
98     True
99     True
Name: final, Length: 99, dtype: bool

In [19]:
# This is pretty fundamental, so take a moment to look at this. The result of broadcasting a comparison
# operator is a Boolean mask - true or false values depending upon the results of the comparison. Underneath,
# pandas is applying the comparison operator you specified through vectorization (so efficiently and in
# parallel) to all of the values in the array you specified which, in this case, is the final
# column of the dataframe. The result is a series, since only one column is being operator on, filled with
# either True or False values, which is what the comparison operator returns.

In [20]:
# So, what do you do with the boolean mask once you have formed it? Well, you can just lay it on top of the
# data to "hide" the data you don't want, which is represented by all of the False values. We do this by using
# the .where() function on the original DataFrame.
df.where(nPassed_Exam).head()

,assignment,tutorial,midterm,takehome,final
Serial_Numb,,,,,
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,49.38,105.93,80.56
5,91.32,93.64,95.00,107.41,73.89


In [21]:
# We see that the resulting data frame keeps the original indexed values, and only data which met
# the condition was retained. All of the rows which did not meet the condition have NaN data instead,
# but these rows were not dropped from our dataset.
#
# The next step is, if we don't want the NaN data, we use the dropna() function
df.where(nPassed_Exam).dropna().head(15)

,assignment,tutorial,midterm,takehome,final
Serial_Numb,,,,,
5,91.32,93.64,95.00,107.41,73.89
14,91.28,83.53,81.25,99.81,92.22
24,80.44,90.20,46.25,91.48,72.22
26,91.28,95.24,82.50,97.59,92.78
27,82.45,86.65,93.12,85.56,89.17
29,91.32,94.89,76.25,107.41,85.56
34,98.58,91.03,101.25,104.26,107.78
40,95.00,94.36,89.38,100.93,85.00
42,95.14,82.67,110.00,89.81,90.83


In [22]:
df

,assignment,tutorial,midterm,takehome,final
Serial_Numb,,,,,
1,57.14,34.09,64.38,51.48,52.50
2,95.05,105.49,67.50,99.07,68.33
3,83.70,83.17,NaN,63.15,48.89
4,NaN,NaN,49.38,105.93,80.56
5,91.32,93.64,95.00,107.41,73.89
...,...,...,...,...,...
95,NaN,103.71,45.00,93.52,61.94
96,NaN,80.54,41.25,93.70,39.72
97,89.94,102.77,87.50,90.74,87.78


In [23]:
# lets talk about combining multiple boolean masks, such as multiple criteria for
# including.  & is the bit masking in computer science

In [24]:

(df['final'] > 70) & (df['final'] < 95)

Serial_Numb
1     False
2     False
3     False
4      True
5      True
      ...  
95    False
96    False
97     True
98     True
99     True
Name: final, Length: 99, dtype: bool

In [25]:
# Another way to do this is to just get rid of the comparison operator completely, and instead
# use the built in functions which mimic this approach
df['final'].gt(70) & df['final'].lt(95)

Serial_Numb
1     False
2     False
3     False
4      True
5      True
      ...  
95    False
96    False
97     True
98     True
99     True
Name: final, Length: 99, dtype: bool

In [26]:
# These functions are build right into the Series and DataFrame objects, so you can chain them
# too, which results in the same answer and the use of no visual operators. You can decide what
# looks best for you
df['final'].gt(70).lt(90)

Serial_Numb
1     True
2     True
3     True
4     True
5     True
      ... 
95    True
96    True
97    True
98    True
99    True
Name: final, Length: 99, dtype: bool

In [27]:
# This only works if your operator, such as less than or greater than, is built into the DataFrame, but I
# certainly find that last code example much more readable than one with ampersands and parenthesis.